In [46]:
import pandas as pd

data = pd.read_csv('new_player_visualization.csv', sep=',')
data = data.dropna()
data

,Unnamed: 0.1,Unnamed: 0,player,player_id,yearID,pos,WAR,length,amount,out_of_contract,total_WAR
0,0,2,Jose Abreu,abreujo02,2023.0,h,0.0,3.0,58500000.0,False,32.0
1,1,3,Jose Abreu,abreujo02,2022.0,h,4.3,3.0,58500000.0,True,32.0
2,2,4,Jose Abreu,abreujo02,2021.0,h,2.9,3.0,58500000.0,True,32.0
3,3,5,Jose Abreu,abreujo02,2020.0,h,3.0,3.0,58500000.0,True,32.0
4,4,6,Jose Abreu,abreujo02,2019.0,h,2.5,3.0,58500000.0,True,32.0
...,...,...,...,...,...,...,...,...,...,...,...
5800,5797,41290,Bo Jackson,jacksbo01,1990.0,h,3.5,1.0,2375000.0,True,8.4
5801,5798,41291,Bo Jackson,jacksbo01,1989.0,h,2.7,1.0,2375000.0,True,8.4
5802,5799,41292,Bo Jackson,jacksbo01,1988.0,h,1.7,1.0,2375000.0,True,8.4
5803,5800,41293,Bo Jackson,jacksbo01,1987.0,h,0.0,1.0,2375000.0,True,8.4


In [47]:
amount_filter = data
# contracts with amount between 100_000_000 and 200_000_000
amount_filter = amount_filter[(amount_filter['amount'] > 200_000_000)]
data = amount_filter

In [48]:
from scipy import stats

# Group the data by 'player_id'
grouped = data.groupby('player_id')

In [49]:
# Initialize a dictionary to store the t-test results for each player
t_test_results = {}

# Perform the t-test for each player
for player_id, group in grouped:
    group1 = group[group['out_of_contract'] == True]['WAR']
    group2 = group[group['out_of_contract'] == False]['WAR']

    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy='omit')
    t_test_results[player_id] = (t_statistic, p_value)

# Print the t-test results
not_significant = {}
significant = {}
very_significant = {}
highly_significant = {}


for player_id, (t_statistic, p_value) in t_test_results.items():
    if p_value > 0.05:
        not_significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.5 and p_value > 0.01:
        significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.01 and p_value > 0.001:
        very_significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.001 and p_value > 0.0001:
        highly_significant[player_id] = (t_statistic, p_value)
    
    else:
        pass

print('Not significant: ' + str(round(len(not_significant)/len(t_test_results)*100, 2)) + '%')
print('Significant: ' + str(round(len(significant)/len(t_test_results)*100, 2)) + '%')
print('Very significant: ' + str(round(len(very_significant)/len(t_test_results)*100, 2)) + '%')
print('Highly significant: ' + str(round(len(highly_significant)/len(t_test_results)*100, 2)) + '%')

print(len(not_significant) + len(significant) + len(very_significant) + len(highly_significant))

Not significant: 91.67%
Significant: 4.17%
Very significant: 2.08%
Highly significant: 0.0%
47


In [50]:
from matplotlib import pyplot as plt
group1 = data[data['out_of_contract'] == True]['WAR']
group2 = data[data['out_of_contract'] == False]['WAR']

t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy='omit')

print(f"T-statistic: {t_statistic}, P-value: {p_value}")

T-statistic: 0.4097312486784974, P-value: 0.682146948093737


In [51]:
t_test_groups = [not_significant, significant, very_significant, highly_significant]
grrr = 0
rawrrr = 0
for group in t_test_groups:
    total_amount = 0
    total_length = 0
    for name in group:
        tables = data[data['player_id'] == name]
        amount = tables['amount'].iloc[0]
        length = tables['length'].iloc[0]
       
        per_year = amount/length
        per_year = '${:,.2f}'.format(per_year)
        #print(tables['player'].iloc[0] + ' made ' + str(per_year))

        total_amount += amount
        total_length += length

        avg = total_amount/total_length
        avg = '${:,.2f}'.format(avg)
    grrr += total_amount
    rawrrr += total_length
    print(avg + ' is the average amount per year for the group')

grrr = '${:,.2f}'.format(grrr/rawrrr)

print('')
print(grrr)


    
    

$20,003,389.83 is the average amount per year for the group
$25,500,000.00 is the average amount per year for the group
$12,888,888.89 is the average amount per year for the group
$12,888,888.89 is the average amount per year for the group

$20,044,025.16


In [52]:

for name in significant:
    tables = data[data['player_id'] == name]
    amount = tables['amount'].iloc[0]
    length = tables['length'].iloc[0]
    per_year = amount/length
    per_year = '${:,.2f}'.format(per_year)
    print(tables['player'].iloc[0] + ' made ' + str(per_year))

Kris Bryant made $26,000,000.00
Marcus Semien made $25,000,000.00


In [53]:

for name in very_significant:
    tables = data[data['player_id'] == name]
    amount = tables['amount'].iloc[0]
    length = tables['length'].iloc[0]
    per_year = amount/length
    per_year = '${:,.2f}'.format(per_year)
    print(tables['player'].iloc[0] + ' made ' + str(per_year))

Ken Griffey Jr made $12,888,888.89


In [54]:

for name in highly_significant:
    tables = data[data['player_id'] == name]
    amount = tables['amount'].iloc[0]
    length = tables['length'].iloc[0]
    per_year = amount/length
    per_year = '${:,.2f}'.format(per_year)
    print(tables['player'].iloc[0] + ' made ' + str(per_year))

In [55]:

for name in not_significant:
    tables = data[data['player_id'] == name]
    amount = tables['amount'].iloc[0]
    length = tables['length'].iloc[0]
    per_year = amount/length
    per_year = '${:,.2f}'.format(per_year)
    print(tables['player'].iloc[0] + ' made ' + str(per_year))

Ronald Acuna Jr made $12,500,000.00
Jose Altuve made $30,200,000.00
Yordan Alvarez made $19,166,666.67
Elvis Andrus made $15,000,000.00
Javier Baez made $23,333,333.33
Carlos Beltran made $17,000,000.00
Ryan Braun made $21,000,000.00
Alex Bregman made $20,000,000.00
Byron Buxton made $14,285,714.29
Nick Castellanos made $20,000,000.00
Yoenis Cespedes made $27,500,000.00
Shin-Soo Choo made $18,571,428.57
Carl Crawford made $20,285,714.29
Chris Davis made $23,000,000.00
Jacoby Ellsbury made $21,857,142.86
Freddie Freeman made $27,000,000.00
Jason Giambi made $17,142,857.14
Paul Goldschmidt made $26,000,000.00
Adrian Gonzalez made $22,000,000.00
Josh Hamilton made $25,000,000.00
Jason Heyward made $23,000,000.00
Matt Holliday made $17,142,857.14
Ryan Howard made $25,000,000.00
Derek Jeter made $18,900,000.00
Matt Kemp made $20,000,000.00
Evan Longoria made $16,666,666.67
JD Martinez made $22,000,000.00
Joe Mauer made $23,000,000.00
Brandon Nimmo made $20,250,000.00
Matt Olson made $21,000